### server.js

```javascript
import express from "express";
import http from "http";
import SocketIO from "socket.io";

const app = express();
const port = 3000;

// json file 입출력 설정
app.use(express.json());

// pug 설정
app.set("view engine", "pug");
app.set("views", __dirname + "/views");

// static 작업
app.use("/public", express.static(__dirname + "/public"));

// route 설정
app.get("/", (_, res) => res.render("home"));
app.get("/*", (_, res) => res.redirect("/"));

// http server
const httpServer = http.createServer(app);

// io server
const wsServer = SocketIO(httpServer);

wsServer.on("connection", (socket) => {
  socket["nickname"] = "Anon";
  socket.onAny((event) => {
    console.log(`Socket Event: ${event}`);
  });

  socket.on("enter_room", (roomName, done) => {
    socket.join(roomName);
    done();
    // room 전체에 메시지 보내기
    socket.to(roomName).emit("welcome", socket.nickname);
  });

  socket.on("disconnecting", () => {
    socket.rooms.forEach((room) =>
      socket.to(room).emit("bye", socket.nickname)
    );
  });

  socket.on("message", (msg, room, done) => {
    socket.to(room).emit("message", `${socket.nickname} : ${msg}`);
    done();
  });

  socket.on("nickname", (nickname) => (socket["nickname"] = nickname));
});

httpServer.listen(port, () => {
  console.log(`Joom app listening on http://localhost:${port}`);
});

```

### app.js
```javascript
const socket = io();

const welcome = document.querySelector("#welcome");
const form = welcome.querySelector("form");
const room = document.getElementById("room");

const msgForm = document.getElementById("msg");
const nameForm = document.getElementById("name");

room.hidden = true;

let roomName;

function addMessage(message) {
  const ul = room.querySelector("ul");
  const li = document.createElement("li");
  li.innerText = message;
  ul.appendChild(li);
}

form.addEventListener("submit", (event) => {
  event.preventDefault();
  const input = form.querySelector("input");
  socket.emit("enter_room", input.value, () => {
    welcome.hidden = true;
    room.hidden = false;
    const h3 = room.querySelector("h3");
    h3.innerText = `Room ${roomName}`;
  });
  roomName = input.value;
  input.value = "";
});

msgForm.addEventListener("submit", (event) => {
  event.preventDefault();
  const roomInput = msgForm.querySelector("input");
  socket.emit("message", roomInput.value, roomName, () => {
    addMessage(`You: ${roomInput.value}`);
    roomInput.value = "";
    roomInput.focus();
  });
});

nameForm.addEventListener("submit", (event) => {
  event.preventDefault();
  const nameInput = nameForm.querySelector("input");
  socket.emit("nickname", nameInput.value);
  nameForm.hidden = true;
});

socket.on("welcome", (user) => {
  addMessage(`${user} joined!`);
});

socket.on("bye", (user) => {
  addMessage(`${user} left`);
});

socket.on("message", addMessage);

```

### home.pug
```pug
doctype html
html(lang="en")
  head
    meta(charset="UTF-8")
    meta(http-equiv="X-UA-Compatible", content="IE=edge")
    meta(name="viewport", content="width=device-width, initial-scale=1.0")
    title Joom
    //- MVP CSS 사용
    link(rel="stylesheet", href="https://unpkg.com/mvp.css@1.12/mvp.css") 
  body 
    header
      h1 Joom 
    main 
      div#welcome
        form 
          input(placeholder="room name", required, type='text')
          button Enter Room
      div#room
        h3 
        ul
        form#name 
          input(placeholder="nickname", required, type='text')
          button Save 

        form#msg
          input(placeholder="message", required, type='text')
          button Send


    script(src="/socket.io/socket.io.js")
    script(src="/public/js/app.js") 
```